In [25]:
import os

import matplotlib.pyplot as plt
import pandas as pd

In [26]:
# Função para carregar e processar os dados de cada modelo
def process_model_data(csv_path, model_name):
    data = pd.read_csv(csv_path)
    # Adiciona a identificação do modelo nos dados
    data["model"] = model_name
    return data

# Função para calcular métricas estatísticas
def calculate_metrics(data, metrics, model_name):
    stats = {}
    for metric in metrics:
        stats[f"{metric}_mean"] = data[metric].mean()
        stats[f"{metric}_std"] = data[metric].std()
        stats[f"{metric}_max"] = data[metric].max()
        stats[f"{metric}_min"] = data[metric].min()
    stats["model"] = model_name
    return stats

# Função para salvar métricas como CSV
def save_metrics_as_csv(metrics, filename):
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(filename, index=False)
    print(f"Métricas salvas em: {filename}")

# Função para gerar e salvar gráficos
def save_metric_plot(data_general, data_subdivided, metric, ylabel, title, filename):
    plt.figure(figsize=(10, 6))
    plt.plot(data_general["epoch"], data_general[metric], label="Modelo Geral")
    plt.plot(data_subdivided["epoch"], data_subdivided[metric], label="Modelo Subdividido", linestyle='--')
    plt.xlabel("Época")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    print(f"Gráfico salvo em: {filename}")
    plt.close()

In [27]:

from pathlib import Path


# Função principal
def main():
    def get_path_from_cwd(x): return Path(os.path.join(os.getcwd(), "..", x)).resolve()

    # Configurações de arquivos
    general_csv = get_path_from_cwd("runs/detect/train23/results.csv")
    # subdivided_csv = get_path_from_cwd("runs/detect/train21/results.csv")
    subdivided_csv = get_path_from_cwd("runs/detect/train30/results.csv")
    output_folder = get_path_from_cwd("experiments/comparison")

    os.makedirs(output_folder, exist_ok=True)

    # Carregar os dados
    data_general = process_model_data(general_csv, "Geral")
    data_subdivided = process_model_data(subdivided_csv, "Subdividido")

    # Definir métricas de interesse
    metrics = ["metrics/mAP50(B)", "metrics/mAP50-95(B)", "metrics/precision(B)", "metrics/recall(B)",
               "train/box_loss", "train/cls_loss", "val/box_loss", "val/cls_loss"]

    # Calcular estatísticas gerais para cada modelo
    general_metrics = calculate_metrics(data_general, metrics, "Geral")
    subdivided_metrics = calculate_metrics(data_subdivided, metrics, "Subdividido")

    # Salvar estatísticas em CSV
    save_metrics_as_csv(general_metrics, os.path.join(output_folder, "general_metrics_summary.csv"))
    save_metrics_as_csv(subdivided_metrics, os.path.join(output_folder, "subdivided_metrics_summary.csv"))

    # Gerar gráficos e salvar como PNG
    for metric in metrics:
        ylabel = metric.split("/")[-1]  # Nome mais simples para o eixo Y
        plot_title = f"Comparação de {ylabel} entre os Modelos"
        plot_filename = os.path.join(output_folder, f"{metric.replace('/', '_')}_comparison.png")
        save_metric_plot(data_general, data_subdivided, metric, ylabel, plot_title, plot_filename)

    # Salvar métricas finais da última época
    general_final_metrics = data_general.iloc[-1][metrics].to_dict()
    subdivided_final_metrics = data_subdivided.iloc[-1][metrics].to_dict()
    final_metrics = pd.DataFrame([general_final_metrics, subdivided_final_metrics])
    final_metrics["model"] = ["Geral", "Subdividido"]
    final_metrics.to_csv(os.path.join(output_folder, "final_metrics_comparison.csv"), index=False)
    print(f"Métricas finais salvas em: {os.path.join(output_folder, 'final_metrics_comparison.csv')}")

In [28]:

if __name__ == "__main__":
    main()

/mnt/c/Users/Ricardo/Documents/Development/tcc-1/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


Métricas salvas em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/general_metrics_summary.csv
Métricas salvas em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/subdivided_metrics_summary.csv
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_mAP50(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_mAP50-95(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_precision(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_recall(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/train_box_loss_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/train_cls_loss_comparison.png
Gráfico salvo em: /mn

In [29]:
import os
import re
import pandas as pd
import yaml

base_dir = "../runs/detect"
pattern = r"train(\d+)$"  # Regex para achar 'train' seguido de um número
best_results = []

# 1. Listar todos os diretórios em runs/detect e filtrar por trainNN
for run_name in os.listdir(base_dir):
    match = re.match(pattern, run_name)
    if not match:
        continue

    run_index = int(match.group(1))

    # 2. Filtrar apenas runs com índice maior que 30
    if run_index <= 40:
        continue

    run_path = os.path.join(base_dir, run_name)

    # Localiza o arquivo args.yaml
    args_path = os.path.join(run_path, "args.yaml")
    if not os.path.isfile(args_path):
        print(f"[AVISO] args.yaml não encontrado em {run_path}.")
        continue

    # Lê o arquivo args.yaml para extrair o nome do modelo
    with open(args_path, "r") as f:
        args_data = yaml.safe_load(f)
        model_name = args_data.get("model", "modelo_desconhecido")

    # Localiza o arquivo results.csv
    csv_path = os.path.join(run_path, "results.csv")
    if not os.path.isfile(csv_path):
        print(f"[AVISO] results.csv não encontrado em {run_path}.")
        continue

    # Lê o CSV em um DataFrame do pandas
    df = pd.read_csv(csv_path)

    # Verifica se há a coluna de mAP50-95
    if "metrics/mAP50-95(B)" not in df.columns:
        print(
            f"[AVISO] Coluna 'metrics/mAP50-95(B)' não encontrada em {csv_path}.")
        continue

    # Verifica se existe algum valor não-nulo nessa coluna
    if df["metrics/mAP50-95(B)"].dropna().empty:
        print(
            f"[AVISO] Nenhum valor válido (não-nulo) em 'metrics/mAP50-95(B)' em {run_path}.")
        continue

    # Identifica o índice com o maior valor de mAP50-95
    best_index = df["metrics/mAP50-95(B)"].idxmax()
    best_row = df.loc[best_index]

    # Salva resultados básicos (ajuste conforme desejar)
    best_results.append({
        "run": run_name,
        "model": model_name,
        "epoch": int(best_row["epoch"]),
        "mAP50-95": best_row["metrics/mAP50-95(B)"]
    })

# Converte para DataFrame e ordena pelo melhor mAP50-95
df_best = pd.DataFrame(best_results)
if not df_best.empty:
    df_best.sort_values(by="mAP50-95", ascending=False, inplace=True)

print(df_best)

[AVISO] results.csv não encontrado em ../runs/detect/train49.
[AVISO] results.csv não encontrado em ../runs/detect/train50.
       run        model  epoch  mAP50-95
7  train48   yolo11m.pt    226   0.74094
9  train52   yolo11m.pt    226   0.74094
0  train41   yolov9m.pt    384   0.66601
2  train43  yolov10m.pt    240   0.65493
3  train44   yolov5m.pt    176   0.65298
4  train45   yolov3u.pt    220   0.64664
6  train47   yolo11m.pt    311   0.61877
8  train51   yolo11m.pt    261   0.60368
5  train46   yolo11m.pt    144   0.53481
1  train42  yolov10m.pt      1   0.45046


In [30]:
def converter_segundos(segundos):
    """Converte um total de segundos para horas, minutos e segundos."""
    horas = segundos // 3600
    resto = segundos % 3600
    minutos = resto // 60
    segundos_restantes = resto % 60
    return horas, minutos, segundos_restantes



def formatar_tempo(horas, minutos, segundos):
    """Formata a saída em português de forma elegante."""
    partes = []
    if horas > 0:
        partes.append(f"{horas} {'hora' if horas == 1 else 'horas'}")
    if minutos > 0:
        partes.append(f"{minutos} {'minuto' if minutos == 1 else 'minutos'}")
    if segundos > 0 or not partes:
        partes.append(
            f"{segundos} {'segundo' if segundos == 1 else 'segundos'}")

    if len(partes) == 1:
        return partes[0]
    else:
        return ", ".join(partes[:-1]) + " e " + partes[-1]

In [32]:
import os
import pandas as pd

# Dicionário que mapeia cada run ao nome de modelo correspondente
runs_of_interest = {
    # "train23": "yolov8m",
    # "train41": "yolov9m",
    # "train42": "yolov10m",
    # "train40": "yolo11m",
    # "train44": "yolov5m",
    # "train45": "yolov3u",


    "train51": "good",
    "train52": "badm",

    # Adicione aqui outros runs e nomes de modelo conforme necessário
}

# Lista para armazenar o melhor resultado de cada run
best_results = []

for run_name, model_name in runs_of_interest.items():
    csv_path = f"../runs/detect/{run_name}/results.csv"

    if not os.path.isfile(csv_path):
        print(f"Arquivo CSV não encontrado para o run {run_name}: {csv_path}")
        continue

    # Lê o CSV em um DataFrame do pandas
    df = pd.read_csv(csv_path)

    # Verifica se a coluna da métrica existe no DataFrame
    if "metrics/mAP50(B)" not in df.columns:
        print(f"Coluna 'metrics/mAP50-95(B)' não encontrada em {csv_path}.")
        continue

    # Verifica se há valores não-nulos para essa métrica
    if df["metrics/mAP50(B)"].dropna().empty:
        print(
            f"Nenhum valor válido (não-nulo) para 'metrics/mAP50-95(B)' em {run_name}.")
        continue

    # Localiza o índice da linha com maior valor de mAP50-95
    best_index = df["metrics/mAP50-95(B)"].idxmax()

    # Seleciona a linha do melhor resultado
    best_row = df.loc[best_index]

    best_results.append({
        "run": run_name,
        "modelo": model_name,
        "épocas": int(best_row["epoch"]),
        "tempo": formatar_tempo(*converter_segundos(best_row["time"])),
        "train/box_loss": best_row["train/box_loss"],
        "train/cls_loss": best_row["train/cls_loss"],
        "train/dfl_loss": best_row["train/dfl_loss"],
        "precision": best_row["metrics/precision(B)"],
        "recall": best_row["metrics/recall(B)"],
        "mAP50": best_row["metrics/mAP50(B)"],
        "mAP50-95": best_row["metrics/mAP50-95(B)"]
    })

# Converte a lista de dicionários em um DataFrame
df_best = pd.DataFrame(best_results)

# Ordena pelos melhores resultados em mAP50-95 (decrescente)
df_best.sort_values(by="mAP50-95", ascending=False, inplace=True)

# Exemplo de geração de tabela em LaTeX usando pandas
# Personalize as colunas e formatação conforme desejar
latex_table_content = df_best.to_latex(
    index=False,
    float_format="%.4f",
    columns=[
        # "run",
        "modelo",
        "épocas",
        "tempo",
        # "train/box_loss",
        # "train/cls_loss",
        # "train/dfl_loss",
        # "precision",
        # "recall",
        "mAP50",
        "mAP50-95"
    ]
)

# Para incluir ambiente LaTeX com caption e label
latex_table = r"""
\begin{table}[ht]
\centering
""" + latex_table_content + r"""
\caption{Resultados dos melhores modelos para cada run.}
\label{tab:resultados}
\end{table}
"""

print(latex_table)


\begin{table}[ht]
\centering
\begin{tabular}{lrlrr}
\toprule
modelo & épocas & tempo & mAP50 & mAP50-95 \\
\midrule
badm & 226 & 1.0 hora, 50.0 minutos e 42.27000000000044 segundos & 0.9631 & 0.7409 \\
good & 261 & 4.0 horas, 33.0 minutos e 9.900000000001455 segundos & 0.9153 & 0.6037 \\
\bottomrule
\end{tabular}

\caption{Resultados dos melhores modelos para cada run.}
\label{tab:resultados}
\end{table}

